In [23]:
import plac
import random
import pathlib
import cytoolz
import numpy
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.layers import TimeDistributed
from keras.optimizers import Adam
import thinc.extra.datasets
from spacy.compat import pickle
import spacy
from textblob import TextBlob
import numpy as np

In [2]:
def read_data(data_dir, limit=0):
    examples = []
    for subdir, label in (('pos', 1), ('neg', 0)):
        for filename in (data_dir / subdir).iterdir():
            with filename.open() as file_:
                text = file_.read()
            examples.append((text, label))
    random.shuffle(examples)
    if limit >= 1:
        examples = examples[:limit]
    return zip(*examples) # Unzips into two lists

In [3]:
imdb_data = thinc.extra.datasets.imdb()

In [25]:
(imdb_data[1][1])

('John Travolta was excellent as "Michael" in the movie by the same name. I don\'t think a better portrayal could have been done. The movie was funny, yet touching. Michael is a very "human angel" (If their is such). Andie MacDowell is superb in her role as a reporter, as she goes from disbelief to belief. Bill Hurt and Robert Pastorelli are great as fellow reporters, each bent on proving the hoax of the angel on earth. Each of the supporting cast is wonderful, especially the older woman (I do not know her screen name), who plays Michael\'s mother! One of the cutest movies I have seen in years... I could watch this movie dozens of times!',
 1)

In [26]:
blobs = TextBlob(imdb_data[1][1][0])

In [62]:
polarity = [sentence.sentiment.polarity for sentence in blobs.sentences]
polarity

[-0.5, 0.0, 0.0, 0.0, 0.0]

In [32]:
polarity = []
for sentence in blobs.sentences:
    if sentence.sentiment.polarity:
        polarity.append(sentence.sentiment.polarity)

In [33]:
np.mean(polarity)

0.5899305555555555

In [58]:
def evaluate(data):
    print("Processing %d..." % len(data))
    correct = 0
    i = 0
    for text in data:
        doc = text[0]
        label = text[1]
        blobs = TextBlob(doc)
        polarity = []
        for sentence in blobs.sentences:
            polarity.append(sentence.sentiment.polarity)
        sentiment = np.mean(polarity)
        correct += bool(sentiment >= 0.05) == bool(label)
        i += 1
    return float(correct) / i

In [59]:
evaluate(imdb_data[1])

Processing 25000...


0.75144

In [65]:
sentences = ["Im feeling quite sad today",
            "I am not feeling well",
            "i have a brother and a sister",
            "i work as an engineer at google",
            "im quite depressed",
            "i dont like this",
            "i want to kill myself",
            "i hate you"]

sentences = '. '.join(sentences)

In [74]:
blobs = TextBlob(sentences)

polarity = []
for sentence in blobs.sentences:
    print(sentence, sentence.sentiment.polarity)
    if sentence.sentiment.polarity:
        polarity.append(sentence.sentiment.polarity)

Im feeling quite sad today. -0.5
I am not feeling well. 0.0
i have a brother and a sister. 0.0
i work as an engineer at google. 0.0
im quite depressed. 0.0
i dont like this. 0.0
i want to kill myself. 0.0
i hate you -0.8
